In [38]:
from pysmt.shortcuts import *
import pysmt.typing as types
import numpy as np

n = 10     # dimensão do espaço de amostras
m = 64     # numero de amostras
p = 1

# Escolha de "solver"
#name = "msat"
name = "z3"

# Funções auxiliares para BitVec's
def bv_rn():       # gera pseudo-aleatoriamente um BitVec para representar o autómato
    I = 0
    c_x = np.random.randint(1, n-1)
    c_y = np.random.randint(1, n-1)
    for i in range(n):
        I += np.random.binomial(1,p) * 2**i
    for j in range(n):
        I += np.random.binomial(1,p) * 2**(n*j)
    for i in range(c_x-1, c_x+2):
        for j in range(c_y-1, c_y+2):
            I += 2**(n*j+i)
    print(I)
    return BV(I,n*n)

b = bv_rn()
print(b)

def bv_sel(z,i,j):                    # seleciona o bit (i,j) do BitVec "z"
    return BVZExt(BVExtract(z,start=i+j*n,end=i+j*n),n*n)

print(bv_sel(b,0,0))

"""# Variáveis e Problema
x = Symbol('x',types.BVType(n))
e = Symbol('e',types.BVType(n))
problem = And([(bv_hamm(w,x) <= e)  for w in W])

# Solução e Pseudo Optimização
with Solver(name=name) as solver:
    solver.add_assertion(problem)
    for t in range(1,n):
        solver.push()
        solver.add_assertion(e <= t)
        if solver.solve():
            print(solver.get_model())
            break
        solver.pop()"""

1239282502365493782184462336
1239282502365493782184462336_100
(1239282502365493782184462336_100[0:0] ZEXT 100)


"# Variáveis e Problema\nx = Symbol('x',types.BVType(n))\ne = Symbol('e',types.BVType(n))\nproblem = And([(bv_hamm(w,x) <= e)  for w in W])\n\n# Solução e Pseudo Optimização\nwith Solver(name=name) as solver:\n    solver.add_assertion(problem)\n    for t in range(1,n):\n        solver.push()\n        solver.add_assertion(e <= t)\n        if solver.solve():\n            print(solver.get_model())\n            break\n        solver.pop()"